# Obtaining Our Data - Lab

## Introduction
In this lab you'll practice your munging and transforming skills in order to load in your data to solve a regression problem.

## Objectives
You will be able to:
* Understand the ETL process and the steps it consists of
* Understand the challenges of working with data from multiple sources 

## Task Description

Your boss gives you a general description of some of the datasets at your disposal for analyzing weekly store sales. They're eventually looking for you to build a model to help determine what factors impact sales, and model future sales forecasting for business planning.  
  
Most of the properietary store data sits in the company sql database, accessible by all managers and above. The database is called **Walmart.db** Your boss provides you with the following basic schema:  

<img src='db_schema.jpg' width=500>  

She then tells you that she's put together a second dataset on general economy statistics for the various dates that she would also like you to incorporate in your analysis. That data, she says, is stored in a file **economy_data.csv**.

As a first step in creating your model for providing recommendations and projections, load and synthesize these disperate datasets into a singular unified DataFrame. Then save your results to a file **Merged_Store_Data.csv**.

Make sure you check the various data types and merge appropriately.

In [1]:
import sqlite3
import pandas as pd

In [2]:
# Create connection:
con = sqlite3.connect('Walmart.db')
# Create cursor:
cur = con.cursor()

In [5]:
# Select data:
cur.execute("""SELECT * FROM sales JOIN store_details USING(store);""")
wal_df = pd.DataFrame(cur.fetchall())
wal_df.head(3)

,0,1,2,3,4,5,6
0,1,1,2010-02-05,24924.50,False,A,151315
1,1,1,2010-02-12,46039.49,True,A,151315
2,1,1,2010-02-19,41595.55,False,A,151315


In [10]:
wal_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452192 entries, 0 to 452191
Data columns (total 7 columns):
0    452192 non-null int64
1    452192 non-null int64
2    452192 non-null object
3    452192 non-null float64
4    452192 non-null object
5    452192 non-null object
6    452192 non-null int64
dtypes: float64(1), int64(3), object(3)
memory usage: 24.1+ MB


In [15]:
# Setting Column Titles:
wal_df.columns = [i[0] for i in cur.description]
wal_df.head(3)

,Store,Dept,Date,Weekly_Sales,IsHoliday,Type,Size
0,1,1,2010-02-05,24924.50,False,A,151315
1,1,1,2010-02-12,46039.49,True,A,151315
2,1,1,2010-02-19,41595.55,False,A,151315


In [17]:
wal_df.shape

(452192, 7)

In [19]:
wal_df2 = pd.read_csv('economy_data.csv')
wal_df2.head(3)

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False


In [20]:
wal_df2.shape

(8190, 12)

In [21]:
mergedf1 = pd.merge(wal_df, wal_df2)
print(mergedf1.shape)
mergedf1.head(3) # Didn't get the results we wanted from this attempt.

(0, 16)


,Store,Dept,Date,Weekly_Sales,IsHoliday,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment


In [22]:
print(wal_df.info())
print('\n')
print(wal_df2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452192 entries, 0 to 452191
Data columns (total 7 columns):
Store           452192 non-null int64
Dept            452192 non-null int64
Date            452192 non-null object
Weekly_Sales    452192 non-null float64
IsHoliday       452192 non-null object
Type            452192 non-null object
Size            452192 non-null int64
dtypes: float64(1), int64(3), object(3)
memory usage: 24.1+ MB
None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 12 columns):
Store           8190 non-null int64
Date            8190 non-null object
Temperature     8190 non-null float64
Fuel_Price      8190 non-null float64
MarkDown1       4032 non-null float64
MarkDown2       2921 non-null float64
MarkDown3       3613 non-null float64
MarkDown4       3464 non-null float64
MarkDown5       4050 non-null float64
CPI             7605 non-null float64
Unemployment    7605 non-null float64
IsHoliday       8190 non-nu

In [23]:
# Identifying shared columns:
shared = [col for col in wal_df.columns if col in wal_df2.columns]
shared

['Store', 'Date', 'IsHoliday']

In [24]:
# Checking data format and resolving differences:
for col in shared:
    c1 = wal_df[col].iloc[0]
    c2 = wal_df2[col].iloc[0]
    print(col)
    print('Types:')
    print('wal_df: {}, wal_df2: {}'.format(type(c1), type(c2)))
    print('\n')
    

Store
Types:
wal_df: <class 'numpy.int64'>, wal_df2: <class 'numpy.int64'>


Date
Types:
wal_df: <class 'str'>, wal_df2: <class 'str'>


IsHoliday
Types:
wal_df: <class 'str'>, wal_df2: <class 'numpy.bool_'>




In [25]:
# IsHoliday should probably be a boolean instead of a string.
wal_df.IsHoliday = wal_df.IsHoliday.astype(bool)

In [26]:
# Retrying merge:
mergedf1 = pd.merge(wal_df, wal_df2)
print(mergedf1.shape)
mergedf1.head(3) 

(31817, 16)


,Store,Dept,Date,Weekly_Sales,IsHoliday,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
0,1,1,2010-02-12,46039.49,True,A,151315,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.24217,8.106
1,1,2,2010-02-12,44682.74,True,A,151315,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.24217,8.106
2,1,3,2010-02-12,10887.84,True,A,151315,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.24217,8.106


In [29]:
# Saving merged data:
mergedf1.to_csv('Merged_Store_Data.csv', index=False)

## Summary
Nice work! You're working more and more independently through the workflow, and ensuring data integrity!